# Quick Pytorch Tutorial Followed
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html


This week we realized that pyTorch is common in image segmentation. Since we have already been interested in trying some pytorch, we quickly followed a tutorial after finishing our other notebooks from this weeks biweekly report. This pytorch notebook is somewhat brief and follows very closely the tutorial from https://pytorch.org/tutorials/beginner/basics/data_tutorial.html but will be usefull as reference and experience if we need to read or implement Pytorch in the future reports. 

We realize that we are deep into the semester so learnning a new language from scratch isn't necessarily the advanced content that will count as a large contribution to our project, however we spend time to follow this tutorial (additionally to our other work) so we chose to include what we finished with. 


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)



Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break


Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [7]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308661  [    0/60000]
loss: 2.296880  [ 6400/60000]
loss: 2.277247  [12800/60000]
loss: 2.271322  [19200/60000]
loss: 2.253552  [25600/60000]
loss: 2.242248  [32000/60000]
loss: 2.236268  [38400/60000]
loss: 2.215842  [44800/60000]
loss: 2.196249  [51200/60000]
loss: 2.175578  [57600/60000]
Test Error: 
 Accuracy: 40.8%, Avg loss: 2.165191 

Epoch 2
-------------------------------
loss: 2.175650  [    0/60000]
loss: 2.167795  [ 6400/60000]
loss: 2.112253  [12800/60000]
loss: 2.124066  [19200/60000]
loss: 2.080508  [25600/60000]
loss: 2.038178  [32000/60000]
loss: 2.046382  [38400/60000]
loss: 1.982019  [44800/60000]
loss: 1.968330  [51200/60000]
loss: 1.907998  [57600/60000]
Test Error: 
 Accuracy: 61.2%, Avg loss: 1.901021 

Epoch 3
-------------------------------
loss: 1.930894  [    0/60000]
loss: 1.908491  [ 6400/60000]
loss: 1.789454  [12800/60000]
loss: 1.824669  [19200/60000]
loss: 1.721670  [25600/60000]
loss: 1.679339  [32000/600

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [14]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [16]:
#_____________________

In [17]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [20]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


In [22]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [23]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [24]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [25]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0405,  0.1483, -0.5676,  0.2000,  0.1174, -0.0378,  0.3396, -0.1610,
         -0.0248, -0.5799, -0.2134,  0.1765, -0.2984,  0.4394,  0.1842,  0.8705,
         -0.2114, -0.1468, -0.4084,  0.4248],
        [-0.0683,  0.6206, -0.0936, -0.0703,  0.2096, -0.4388,  0.2340, -0.1953,
         -0.5588, -0.7402, -0.1870, -0.2300,  0.0921,  0.3973,  0.0572,  0.8845,
         -0.3820,  0.0210, -0.3340,  0.9643],
        [ 0.1404,  0.0749, -0.6519, -0.0476,  0.2245, -0.2323,  0.0212, -0.0794,
         -0.1969, -0.3507, -0.1058, -0.1773, -0.1186,  0.6420,  0.4641,  0.6218,
         -0.2368, -0.0170, -0.1025,  0.7868]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0405, 0.1483, 0.0000, 0.2000, 0.1174, 0.0000, 0.3396, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1765, 0.0000, 0.4394, 0.1842, 0.8705, 0.0000, 0.0000,
         0.0000, 0.4248],
        [0.0000, 0.6206, 0.0000, 0.0000, 0.2096, 0.0000, 0.2340, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0921, 0.3973, 0.05

In [26]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [27]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [28]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0145,  0.0195, -0.0024,  ..., -0.0314,  0.0063, -0.0311],
        [ 0.0095, -0.0058,  0.0119,  ..., -0.0023, -0.0312, -0.0185]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0113, -0.0232], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0037,  0.0147,  0.0271,  ..., -0.0165, -0.0402,  0.0064],
        [-0.0353, -0.0081, -0.0232,  ...,  0.0081, -0.0194, -0.0104]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias 

In [29]:
# -*- coding: utf-8 -*-
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 2299.860595703125
3999 540.2581176757812
5999 248.35223388671875
7999 115.27959442138672
9999 56.349639892578125
11999 30.317068099975586
13999 18.44389533996582
15999 13.159984588623047
17999 10.79162311553955
19999 9.694792747497559
21999 9.042831420898438
23999 8.775341033935547
25999 8.703462600708008
27999 8.88264274597168
29999 8.884757995605469
Result: y = 0.003705973271280527 + 0.8544864654541016 x + -0.0011620450532063842 x^2 + -0.09337783604860306 x^3 + 0.00010247391765005887 x^4 ? + 0.00010247391765005887 x^5 ?
